# geoDB user groups

You may define user groups in geoDB, which serve the purpose to allow sharing of collections between project members while not making them publicly available. This notebook will demonstrate how to work with groups.

In [1]:
from xcube_geodb.core.geodb import GeoDBClient

##  Login from any machine

Install xcube geoDB with command:
  
 `conda install xcube_geodb -c conda-forge`


In [20]:
### uncomment if not in managed environment
#client_id=YourID
#client_secret=YourSecret
#geodb = GeoDBClient(client_id=client_id, client_secret=client_secret, auth_mode="client-credentials")

# remove the following two lines when merging into main!
import os

os.environ["GEODB_API_SERVER_URL"] = "https://xcube-geodb.dev.brockmann-consult.de"

## Login in managed environment

The environment is prepared with your user credentials, so you simply can start the client.

In [ ]:
geodb = GeoDBClient()

## Show current user, and the list of groups it is currently member of

In [ ]:
print(geodb.whoami)
geodb.get_my_groups()

## Create a new group, and checking before if it already exists (an error occurs if it does)

In [ ]:
group_name = 'test_group_' + geodb.whoami
group_name = group_name[:63]  # ensure group_name will not be truncated in database
if group_name not in geodb.get_my_groups():
    geodb.create_group(group_name)


## Add a user and see who's part of the group

In [ ]:
geodb.add_user_to_group('geodb_thomas.storm.test', group_name)
geodb.get_group_users(group_name)

In [ ]:
geodb.get_my_groups()

## Create a small test collection to be shared with the group

In [ ]:
import geopandas

properties = {
    "RABA_PID": "float",
    "RABA_ID": "float",
    "D_OD": "date"
}

collection_name = 'land_use'
geodb.create_collection(collection_name, properties, crs=3794, clear=True)
gdf = geopandas.read_file('data/sample/land_use.shp')
m = geodb.insert_into_collection(collection_name, gdf.iloc[:100, :])
str(m)

## Share new collection with group

In [ ]:
m = geodb.publish_collection_to_group(collection_name, group_name)
str(m)

## Show which users or groups have which rights on the collection

In [ ]:
geodb.get_access_rights(collection_name)

## Unpublish collection again

In [ ]:
m = geodb.unpublish_collection_from_group(collection_name, group_name)
str(m)

## Verify that groups does not have access to the collection anymore

In [ ]:
geodb.get_access_rights(collection_name)

## Finally, do a clean-up of this notebook's leftovers

In [ ]:
geodb.drop_collection("land_use")